In [7]:
import pandas as pd
import numpy as np
import warnings

import plotly.graph_objects as go
import plotly.io as pio


warnings.filterwarnings('ignore')

In [8]:
def generate_gray_tones(source):
    # Your existing code here to generate colors based on some condition or data
    colors = []
    
    # Example: Set opacity to 0.2 for all colors
    for i in source:
        if i in (1,3,4):
            colors.append('rgba(49, 162, 227, 0.8)')
        else:
            colors.append('rgba(128, 128, 128, 0.2)')
    
    return colors


In [9]:
def format_value(value):
    if value >= 1000000:
        return f'{value/1000000:.1f}M'
    elif value >= 1000:
        return f'{value/1000:.1f}K'
    else:
        return f'{value:.0f}'

In [10]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\compiled_data_v2.csv'

df = pd.read_csv(folder_path, encoding='utf-8')

df = df[(df['BU'] == 'CSB') 
            & (df['Fiscal Quarter'].isin(['2024-Q1','2024-Q2','2024-Q3'])) 
            & (df['Country'] == 'United States')
            & (df['Audience Type'] == '1PD')]

#df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]

In [11]:
# Fix issue with 'Audience Type Name': 'CRM-1PD\xa0CRM'
df['Audience Type Name'] = df['Audience Type Name'].str.replace('CRM-1PD\xa0CRM', 'CRM-1PD CRM')

# Get Segments Unique Counts
filtered_df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
df_lvl1_seg = filtered_df.groupby(['Audience Type'])['Segment ID'].nunique().reset_index()
df_lvl2_seg = filtered_df.groupby(['Audience Type Name'])['Segment ID'].nunique().reset_index()
df_lvl3_seg = filtered_df.groupby(['Audience Source'])['Segment ID'].nunique().reset_index()
df_lvl4_seg = filtered_df.groupby(['Display Funnel Mapped'])['Segment ID'].nunique().reset_index()
df_lvl1_seg.rename(columns={'Audience Type':'Label', 'Segment ID':'Segments'}, inplace=True)
df_lvl2_seg.rename(columns={'Audience Type Name':'Label', 'Segment ID':'Segments'}, inplace=True)
df_lvl3_seg.rename(columns={'Audience Source':'Label', 'Segment ID':'Segments'}, inplace=True)
df_lvl4_seg.rename(columns={'Display Funnel Mapped':'Label', 'Segment ID':'Segments'}, inplace=True)
segment_df = pd.concat([df_lvl1_seg, df_lvl2_seg, df_lvl3_seg, df_lvl4_seg])
######

test = df.groupby(['Audience Type', 'Audience Type Name', 'Audience Source','Display Funnel Mapped'])['Spend'].sum().reset_index()

test3 = test.groupby(['Audience Type', 'Audience Type Name', 'Audience Source','Display Funnel Mapped'])['Spend'].sum().reset_index()
test3 = test3[(test3['Audience Source'] != '0')&(test3['Display Funnel Mapped'] != '0')]

test1 = test.groupby(['Audience Type','Audience Type Name'])['Spend'].sum().reset_index()
test1 = test1[(test1['Audience Type Name'] != '0')]

test2 = test.groupby(['Audience Type','Audience Type Name','Audience Source'])['Spend'].sum().reset_index()
test2 = test2[(test2['Audience Type Name'] != '0')&(test2['Audience Source'] != '0')]

test4 = test[test['Audience Source'] == '0'].groupby(['Audience Type', 'Audience Type Name', 'Display Funnel Mapped'])['Spend'].sum().reset_index()
test4 = test4[(test4['Audience Type Name'] != '0')&(test4['Display Funnel Mapped'] != '0')]

test1['Level'] = 1
test2['Level'] = 2
test3['Level'] = 4
test4['Level'] = 3

test1.rename(columns={'Audience Type':'Source', 'Audience Type Name':'Target', 'Spend':'Total'}, inplace=True)
test2.rename(columns={'Audience Type Name':'Source', 'Audience Source':'Target', 'Spend':'Total'}, inplace=True)
test3.rename(columns={'Audience Source':'Source', 'Display Funnel Mapped':'Target', 'Spend':'Total'}, inplace=True)
test4.rename(columns={'Audience Type Name':'Source', 'Display Funnel Mapped':'Target', 'Spend':'Total'}, inplace=True)

concat_df = pd.concat([
                       test1[['Source','Target','Total', 'Level']]
                       , test2[['Source','Target','Total', 'Level']]
                       , test3[['Source','Target','Total', 'Level']]
                        , test4[['Source','Target','Total', 'Level']]
                    ], ignore_index=True)
                    

unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))

mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                     , 'Value': [i for i in range(len(unique_values))]})


unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))
    
mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                          , 'Value': [i for i in range(len(unique_values))]})

# Replace values based on the mappings
concat_df['Source'] = concat_df['Source'].replace(mapping_df.set_index('Label')['Value'])
concat_df['Target'] = concat_df['Target'].replace(mapping_df.set_index('Label')['Value'])






# Create a list of columns to process
columns_to_process = ['Audience Type', 'Audience Type Name', 'Audience Source', 'Display Funnel Mapped']

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through each column and calculate sums, percentages, and unique "Segment ID" counts
for column_name in columns_to_process:
    # Calculate sums and percentages
    grouped = df.groupby(column_name)['Spend'].sum().reset_index()
    grouped['Percentage'] = (grouped['Spend'] / grouped['Spend'].sum()) * 100
    grouped.rename(columns={column_name: 'Label'}, inplace=True)
    
    # Calculate unique "Segment ID" counts
    grouped_segment = df[(df['Segment ID'] != '0') & (df['Segment ID'] != 0)].groupby(column_name)['Segment ID'].nunique().reset_index()
    grouped_segment.rename(columns={column_name: 'Label', 'Segment ID': 'Segments'}, inplace=True)
    
    # Merge the two DataFrames
    merged_grouped = grouped.merge(grouped_segment, on='Label', how='left')
    
    # Append the result to result_df
    result_df = pd.concat([result_df, merged_grouped], axis=0, ignore_index=True)

# result_df now contains the combined results
result_df


merged_df = mapping_df.merge(result_df, on='Label', how='left')

merged_df.fillna(0,inplace=True)

merged_df['Label_x'] = (
    merged_df['Label'].astype(str) +
    " (" + merged_df['Segments'].apply(format_value).astype(str) + ") " +
    "<br>" +
    merged_df['Percentage'].apply(lambda x: f'{x:.1f}%').astype(str) +
    " (" + merged_df['Spend'].apply(format_value).astype(str) + ")"
)

############


# Sample data
source = concat_df['Source'].values.tolist()
target = concat_df['Target'].values.tolist()
value = concat_df['Total'].values.tolist()
labels = merged_df['Label_x'].values.tolist()

# List of colors for each link based on some condition or data
colors = generate_gray_tones(source)

# Create links
link = dict(source=source, target=target, value=value, color=colors)

# Create nodes
node = dict(label=labels, pad=30, thickness=20)

# Create a Sankey object
chart = go.Sankey(link=link, node=node, arrangement="snap")

# Build a figure
fig = go.Figure(chart)

# Add a title to the figure
fig.update_layout(title_text='title')

fig.show()


In [12]:
details = df.groupby(['Audience Type', 'Audience Type Name', 'Audience Source','Display Funnel Mapped'])[['Spend','Net Rev']].sum().reset_index()
details['ROAS'] = details['Net Rev'] / details['Spend']
details

,Audience Type,Audience Type Name,Audience Source,Display Funnel Mapped,Spend,Net Rev,ROAS
0,1PD,CATEGORY-1PD CONTEXTUAL,0,High Propensity Prospects,0.000000e+00,0.000000e+00,NaN
1,1PD,CRM-1PD AUDIENCE,0,Demand Creation,0.000000e+00,0.000000e+00,NaN
2,1PD,CRM-1PD CRM,0,Active Prospects,1.434724e+05,4.065587e+05,2.833706
3,1PD,CRM-1PD CRM,0,Brand Seeker/Purchasers,0.000000e+00,0.000000e+00,NaN
4,1PD,CRM-1PD CRM,0,Demand Creation,0.000000e+00,3.277758e+04,inf
5,1PD,CRM-1PD CRM,0,Lead Gen,2.291514e+04,2.962017e+04,1.292603
6,1PD,CRM-1PD CRM,0,No Funnel Specific Budget,0.000000e+00,0.000000e+00,NaN
7,1PD,CRM-1PD CRM,AIQ CRM,Active Prospects,9.506830e+04,2.703776e+04,0.284404
8,1PD,CRM-1PD CRM,Liveramp,Active Prospects,5.906594e+05,1.627788e+06,2.755883
9,1PD,CRM-1PD CRM,Liveramp,High Propensity Prospects,1.337119e+05,7.343052e+04,0.549170
